In [1]:
from qick import *
import matplotlib.pyplot as plt
import numpy as np

import os
import time
from pynq import Overlay, DefaultIP, PL

In [2]:
!pip3 show qickquack

Name: qickquack
Version: 0.0.1
Summary: An extension for qick to use custom DACs
Home-page: https://github.com/hrl-labs-clinic-24-25/qickquack
Author: HMC HRL Clinic Team 24-25
Author-email: UNKNOWN
License: UNKNOWN
Location: /home/xilinx/jupyter_notebooks/qickquack
Requires: qick
Required-by: 


To install qickquack, you MUST run the 'qickquack_installtion' notebook under /jupyter_notebooks/qickquack_dir

In [3]:
import qickquack
help(qickquack)

Help on package qickquack:

NAME
    qickquack

PACKAGE CONTENTS
    AxiPvpGen
    DACs

FILE
    /home/xilinx/jupyter_notebooks/qickquack/qickquack/__init__.py




In [ ]:
from qickquack import *

Need to re-define bitfile_path() so that we can use our new bit file

In [4]:
def _new_bitfile_path():
    """Choose the default firmware path for this board.

    Parameters
    ----------

    Returns
    -------
    str
        absolute path to the firmware bitfile distributed with the QICK library
    """
    #board2file =  {'ZCU216' :'qick_216.bit','ZCU111' :'qick_111.bit','RFSoC4x2' :'qick_4x2.bit'}
    #filename = board2file[os.environ['BOARD']]
    filename = "qick_216_integrated_3.bit"
    #src = os.path.join(os.path.dirname(qick.__file__), filename)
    src = os.path.join("/home/xilinx/bit_files/qick_216_integrated", filename)
    
    print("using the products of hmc big")
    return src

In [5]:
qick.bitfile_path = _new_bitfile_path

In [6]:
#PL.reset()

soc = QickSoc()
print(soc)
soccfg = QickConfig(soc.get_cfg())

using the products of hmc big
QICK running on ZCU216, software version 0.2.325

Firmware configuration (built Wed Apr 23 11:38:34 2025):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	16 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.838 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 0 is 0_228, on JHC1
	1:	axis_signal_gen_v6 - envelope memory 16384 samples (1.709 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 1 is 1_228, on JHC2
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (3.419 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 2 is 2_228, on JHC1
	3:	axis_signal_gen_v6 - envelope memory 16384 samples (1.709 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 3 is 3_228, on JHC2
	4:	axis_sg_mixmux8_v1 - envelope memory 0 samples (0.000 us)
		fs=6881

In [7]:
soc.ip_dict.keys()

dict_keys(['axi_dma_avg', 'axi_dma_buf', 'axi_dma_gen', 'axi_dma_mr', 'axi_dma_tproc', 'axi_intc_0', 'axis_avg_buffer_0', 'axis_avg_buffer_1', 'axis_avg_buffer_2', 'axis_avg_buffer_3', 'axis_avg_buffer_4', 'axis_avg_buffer_5', 'axis_avg_buffer_6', 'axis_avg_buffer_7', 'axis_avg_buffer_8', 'axis_avg_buffer_9', 'ddr4/axis_buffer_ddr_v1_0', 'axis_pfb_readout_v4_0', 'axis_sg_mixmux8_v1_0', 'axis_sg_int4_v2_0', 'axis_sg_int4_v2_1', 'axis_sg_int4_v2_10', 'axis_sg_int4_v2_2', 'axis_sg_int4_v2_3', 'axis_sg_int4_v2_4', 'axis_sg_int4_v2_5', 'axis_sg_int4_v2_6', 'axis_sg_int4_v2_7', 'axis_sg_int4_v2_8', 'axis_sg_int4_v2_9', 'axis_signal_gen_v6_0', 'axis_signal_gen_v6_1', 'axis_signal_gen_v6_2', 'axis_signal_gen_v6_3', 'axis_switch_avg', 'axis_switch_buf', 'axis_switch_ddr', 'axis_switch_gen', 'axis_switch_mr', 'mr_buffer_et_0', 'qick_processor_0', 'clk104_gpio', 'usp_rf_data_converter_0', 'axi_pvp_gen_v7_0', 'zynq_ultra_ps_e_0'])

In [8]:
soc.axi_pvp_gen_v7_0

In [ ]:
class Gvg(asm_v2.AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        self.declare_gen(ch=gen_ch, nqz=cfg['nqz'], mixer_freq=0)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])
       
        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch,
                              outsel='product'
                              )
        self.add_pulse(ch=gen_ch, name="sourcedrain", ro_ch=ro_ch,
                       style="const",
                       freq=cfg['freq'],
                       length=cfg['pulse_len'],
                       phase=0,
                       phrst = 1,
                       gain=cfg['gain'],
                      ) #<-pulse length is most of the dwell time
        self.send_readoutconfig(ch=cfg['ro_ch'], name="myro", t=0)
        self.add_loop("myloop", cfg['loops']) ### just for executing the loop 100 times, this should correspond to the number of steps in one row of the GvG
        
        ### setup QuACK dacs here! ###
        #bias = DAC(soc)
        daccfg = {
            'startvals': [1.5, 2.25],
            'stepsizes': [0.25, 0.25],
            'demuxvals': [3, 4],
            'groups': [0, 1],
            'mode': 0,
            'width': 16,
            'num_dims': 2 #num_dims is the number of groups, NOT the number of dacs (you can have multiple dacs per group)
            }
        
        soc.axi_pvp_gen_v7_0.setup_pvp(cfg = daccfg)
        
        for val in daccfg['demuxvals']:
            bias.init_DAC(val)
            
        soc.axi_pvp_gen_v7_0.start_pvp()
        
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])#<-setup the trigger for the slow DACs, set to pin0 on PMOD0
        self.delay(t=cfg['meas_delay']) # insert a delay to ensure all the registers are set
       
    def _body(self, cfg):
       
      
        self.delay(t=cfg['meas_delay']) # wait a few us before measuring
        self.pulse(ch=cfg['gen_ch'], name="sourcedrain", t=0) # readout pulse
        self.trigger(ros=[cfg['ro_ch']], t=0, pins=[7]) # trigger ADC for readout, pins=[7] also indicates PMOD0 pin trigger, change the index for other pins
        self.delay_auto(t=cfg['meas_delay']) # this can probably be zero, I add in extra timing slack here
 
 
config = {
          'gen_ch': 9,
          'ro_ch': 0,
          'freq': 1,
          'nqz': 1,
          'trig_time': 0,
          'ro_len': 5,
          'pulse_len': 5,
          'gain': 1,
          'meas_delay': 2,
          'loops': 100 #<-number of steps in a single row, in your case upto 256
          }

                            
gvgprog = Gvg(soccfg, reps=1, final_delay=0, cfg=config)
d = gvgprog.acquire(soc)